
# Day 12 · Notebook 2 — Test‑Driven Prompting

**Concept #82 & #83** — *Write the tests first; prompts will follow.*

In traditional software, unit tests catch regressions early. We can do the same for prompts:

1. Specify *expected intent & style* as assertions.  
2. Run the prompt against the LLM with a fixed seed or low temperature.  
3. Refactor until the test passes.  
4. Keep the test in CI so future edits stay green.


In [ ]:

# Install dependencies (comment out if already installed)
# !pip install -q openai pytest tiktoken


In [ ]:

%%bash
cat > prompt_test_suite.py <<'PYTEST'
import pytest, re, hashlib, os
import openai

MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")
API_KEY = os.getenv("OPENAI_API_KEY")  # Make sure this is set in Colab!

def run_prompt(prompt, **params):
    response = openai.ChatCompletion.create(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}],
        temperature=params.get("temperature", 0),
        max_tokens=128,
    )
    return response.choices[0].message.content.strip()

def test_greet_formally():
    """Prompt should always greet with 'Good' and end with exactly one exclamation mark."""
    result = run_prompt("You are a formal greeter. Say hello to Charles.")
    assert re.match(r"Good.*!$", result)

def test_hash_stability():
    """The same prompt+params should have a stable SHA256 digest (deterministic seed)."""
    result = run_prompt("Return the word TEST.", temperature=0)
    digest = hashlib.sha256(result.encode()).hexdigest()
    # Update the expected digest after first green run
    expected = "UPDATE_ME"
    assert digest == expected
PYTEST



### Red/Green Cycle

1. Run `pytest -q prompt_test_suite.py`  
2. Observe failures (“red”)  
3. Modify the prompts or parameters until all tests pass (“green”).  
4. Commit both prompt and tests.



**Exercise:** Replace `UPDATE_ME` with the actual digest after your first successful run, then re‑run the suite to go green.
